In [ ]:
# default_exp utils.matlab_io

In [ ]:
# export
from capetools.imports import *
from capetools.utils.pandas_hacks import reduce_mem_usage
import hdf5storage
import datetime

# Matlab IO
> Reading mat files to python

In [ ]:
#export
def read_mat(filename: Union[str, Path]):
    mat = hdf5storage.loadmat(str(filename))
    return mat

A set of functions to convert `.mat` files from the PopDB to `DataFrames`

In [ ]:
#export
def datetime_mtopy(datenum):
    '''
    Input
        The fractional day count according to datenum datatype in matlab
    Output
        The date and time as a instance of type datetime in python
    Notes on day counting
        matlab: day one is 1 Jan 0000 
        python: day one is 1 Jan 0001
        hence a reduction of 366 days, for year 0 AD was a leap year
    EX:
    df['date'] = df['tvec'].apply(lambda tvec: datetime_mtopy(tvec))
    '''
    ii = datetime.datetime.fromordinal(int(datenum) - 366)
    ff = datetime.timedelta(days=datenum%1)
    return ii + ff 

def get_components(mat):
    "helper function to process the mat file"
    compList = {}
    for name in mat['dataName']:
        compList[name[0][0][0]]=[]
        for item in name[1]:
            compList[name[0][0][0]]+=[x[0][0] for x in item]
    return compList

def dict2tuples(d):
    "helper to split comp names"
    tuples = []
    for key in d:
        tuples += list(zip(['_'.join(key.split())]*len(d[key]), d[key]))
    return tuples

def make_datetime_index(df):
    "creates a datetime index"
    df['date'] = df[0].apply(lambda tvec: datetime_mtopy(tvec))
    df.set_index('date', inplace=True)
    df.drop(columns=[0], axis=1, inplace=True)
    return

def mat2df(filename):
    'Popdb matlab .mat file to pandas df'
    mat = read_mat(filename)
    data = mat['data']
    tvec = mat['tvec']
    datac = np.concatenate((tvec, data), axis=1)
    df = pd.DataFrame(datac)
    compList = get_components(mat)
    compTuples = dict2tuples(compList)
    make_datetime_index(df)
    df.columns = pd.MultiIndex.from_tuples(compTuples, 
                                           names=['component','quantity'])
    return reduce_mem_usage(df)

# Exports -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.core.ipynb.
Converted 01_utils.pandas_hacks.ipynb.
Converted 03_utils.matlab_io.ipynb.
Converted index.ipynb.
